In [1]:
import os

import pandas as pd

In [2]:
summary_list = os.listdir('../data/summaries/')

In [3]:
summary_list

['1 Test - CRO 2003 Democratic Union_summary.txt',
 '0 Calibration - GER 2002 CDU-CSU_summary.txt',
 '0 Calibration - AU 99 FPO_summary.txt',
 '0 Calibration - CZ 2006 Green_summary.txt',
 '1 Test - BUL 2014 Attack_summary.txt',
 '0 Calibration - NL 2003 VVD txt_summary.txt',
 '0 Calibration - HUN 2010 Soc Dem_summary.txt',
 '.gitkeep',
 '0 Calibration - SLE 2014 United Left_summary.txt',
 '0 Calibration - IRL 1989 Lab txt_summary.txt',
 '1 Test - FR 2002 FN txt_summary.txt',
 '1 Test - SWE 2010 Moderate_summary.txt',
 '0 Calibration - UK 2019 Lib Dem_summary.txt',
 '1 Test - POL 2019 Civic Coalition_summary.txt',
 '1 Test - POR1999 Soc Dem_summary.txt',
 '1 Test - NOR 1989 Conservative txt_summary.txt',
 '1 Test - SLO 2006 Xtian Dem Movement_summary.txt']

In [4]:
calibration_summary_list = [x for x in summary_list if 'Calibration' in x]

In [5]:
calibration_summary_list

['0 Calibration - GER 2002 CDU-CSU_summary.txt',
 '0 Calibration - AU 99 FPO_summary.txt',
 '0 Calibration - CZ 2006 Green_summary.txt',
 '0 Calibration - NL 2003 VVD txt_summary.txt',
 '0 Calibration - HUN 2010 Soc Dem_summary.txt',
 '0 Calibration - SLE 2014 United Left_summary.txt',
 '0 Calibration - IRL 1989 Lab txt_summary.txt',
 '0 Calibration - UK 2019 Lib Dem_summary.txt']

In [6]:
ches_results = pd.read_excel('../data/ches_scores/LLM scores vs CHES.xlsx')[['Document','Status','Dimension','Expert mean']]
ches_results = ches_results[~ches_results['Status'].isna()]

In [7]:
ches_results

,Document,Status,Dimension,Expert mean
1,CZ 2006 Green,Calibration,EU,6.25
2,CZ 2006 Green,Calibration,TaxSpend,5.67
3,CZ 2006 Green,Calibration,SocialLifestyle,3
4,CZ 2006 Green,Calibration,Immigration,4.75
5,CZ 2006 Green,Calibration,Environment,x
...,...,...,...,...
69,SWE 2010 Moderate,Test,TaxSpend,7.57
70,SWE 2010 Moderate,Test,SocialLifestyle,3.21
71,SWE 2010 Moderate,Test,Immigration,5.36
72,SWE 2010 Moderate,Test,Environment,5.93


In [8]:
ches_results['Calibration File'] = ches_results['Document'].apply(lambda x: next((f for f in calibration_summary_list if x in f), None))

In [9]:
ches_results['Calibration File'].value_counts()

Calibration File
0 Calibration - CZ 2006 Green_summary.txt           6
0 Calibration - HUN 2010 Soc Dem_summary.txt        6
0 Calibration - IRL 1989 Lab txt_summary.txt        6
0 Calibration - NL 2003 VVD txt_summary.txt         6
0 Calibration - SLE 2014 United Left_summary.txt    6
0 Calibration - UK 2019 Lib Dem_summary.txt         6
Name: count, dtype: int64

In [10]:
ches_results

,Document,Status,Dimension,Expert mean,Calibration File
1,CZ 2006 Green,Calibration,EU,6.25,0 Calibration - CZ 2006 Green_summary.txt
2,CZ 2006 Green,Calibration,TaxSpend,5.67,0 Calibration - CZ 2006 Green_summary.txt
3,CZ 2006 Green,Calibration,SocialLifestyle,3,0 Calibration - CZ 2006 Green_summary.txt
4,CZ 2006 Green,Calibration,Immigration,4.75,0 Calibration - CZ 2006 Green_summary.txt
5,CZ 2006 Green,Calibration,Environment,x,0 Calibration - CZ 2006 Green_summary.txt
...,...,...,...,...,...
69,SWE 2010 Moderate,Test,TaxSpend,7.57,None
70,SWE 2010 Moderate,Test,SocialLifestyle,3.21,None
71,SWE 2010 Moderate,Test,Immigration,5.36,None
72,SWE 2010 Moderate,Test,Environment,5.93,None


In [11]:
issue_map = {'european_union': 'EU',
             'taxation': 'TaxSpend',
             'lifestyle': 'SocialLifestyle',
             'immigration': 'Immigration',
             'environment': 'Environment',
             'decentralization': 'Regions'}

In [12]:
issue_map_reversed = {v: k for k, v in issue_map.items()}

In [13]:
issue_map_reversed

{'EU': 'european_union',
 'TaxSpend': 'taxation',
 'SocialLifestyle': 'lifestyle',
 'Immigration': 'immigration',
 'Environment': 'environment',
 'Regions': 'decentralization'}

In [14]:
ches_results['issue'] = ches_results['Dimension'].apply(lambda x: issue_map_reversed[x])

In [15]:
final_prompt_setup = ches_results[['issue','Calibration File','Expert mean']].where(ches_results['Calibration File'].notna()).dropna()

In [55]:
final_prompt_setup['Expert mean'] = pd.to_numeric(final_prompt_setup['Expert mean'], errors='coerce').round(0).apply(lambda x: str(int(x)) if not pd.isna(x) else 'NA')

In [56]:
final_prompt_setup

,issue,Calibration File,Expert mean
1,european_union,0 Calibration - CZ 2006 Green_summary.txt,6
2,taxation,0 Calibration - CZ 2006 Green_summary.txt,6
3,lifestyle,0 Calibration - CZ 2006 Green_summary.txt,3
4,immigration,0 Calibration - CZ 2006 Green_summary.txt,5
5,environment,0 Calibration - CZ 2006 Green_summary.txt,NA
6,decentralization,0 Calibration - CZ 2006 Green_summary.txt,4
7,european_union,0 Calibration - HUN 2010 Soc Dem_summary.txt,7
8,taxation,0 Calibration - HUN 2010 Soc Dem_summary.txt,4
9,lifestyle,0 Calibration - HUN 2010 Soc Dem_summary.txt,3
10,immigration,0 Calibration - HUN 2010 Soc Dem_summary.txt,4


In [57]:
final_prompt_setup.to_csv('../data/ches_scores/final_prompt_setup.csv', index=False)